# Get Fitbit Data
## Independent variables
### Sleep data

In [1]:
import fitbit
import json
from datetime import datetime, date, timedelta
from ast import literal_eval
import pandas as pd

In [2]:
#!/usr/bin/env python
import cherrypy
import os
import sys
import threading
import traceback
import webbrowser

from urllib.parse import urlparse
from base64 import b64encode
from fitbit.api import Fitbit
from oauthlib.oauth2.rfc6749.errors import MismatchingStateError, MissingTokenError

In [3]:
class OAuth2Server:
    def __init__(self, client_id, client_secret,
                 redirect_uri='http://127.0.0.1:8080/'):
        """ Initialize the FitbitOauth2Client """
        self.success_html = """
            <h1>You are now authorized to access the Fitbit API!</h1>
            <br/><h3>You can close this window</h3>"""
        self.failure_html = """
            <h1>ERROR: %s</h1><br/><h3>You can close this window</h3>%s"""

        self.fitbit = Fitbit(
            client_id,
            client_secret,
            redirect_uri=redirect_uri,
            timeout=10,
        )

        self.redirect_uri = redirect_uri

    def browser_authorize(self):
        """
        Open a browser to the authorization url and spool up a CherryPy
        server to accept the response
        """
        url, _ = self.fitbit.client.authorize_token_url()
        # Open the web browser in a new thread for command-line browser support
        threading.Timer(1, webbrowser.open, args=(url,)).start()

        # Same with redirect_uri hostname and port.
        urlparams = urlparse(self.redirect_uri)
        cherrypy.config.update({'server.socket_host': urlparams.hostname,
                                'server.socket_port': urlparams.port})

        cherrypy.quickstart(self)

    @cherrypy.expose
    def index(self, state, code=None, error=None):
        """
        Receive a Fitbit response containing a verification code. Use the code
        to fetch the access_token.
        """
        error = None
        if code:
            try:
                self.fitbit.client.fetch_access_token(code)
            except MissingTokenError:
                error = self._fmt_failure(
                    'Missing access token parameter.</br>Please check that '
                    'you are using the correct client_secret')
            except MismatchingStateError:
                error = self._fmt_failure('CSRF Warning! Mismatching state')
        else:
            error = self._fmt_failure('Unknown error while authenticating')
        # Use a thread to shutdown cherrypy so we can return HTML first
        self._shutdown_cherrypy()
        return error if error else self.success_html

    def _fmt_failure(self, message):
        tb = traceback.format_tb(sys.exc_info()[2])
        tb_html = '<pre>%s</pre>' % ('\n'.join(tb)) if tb else ''
        return self.failure_html % (message, tb_html)

    def _shutdown_cherrypy(self):
        """ Shutdown cherrypy in one second, if it's running """
        if cherrypy.engine.state == cherrypy.engine.states.STARTED:
            threading.Timer(1, cherrypy.engine.exit).start()


In [4]:
with open('fitbit_tokens.json') as f:
    TOKENS = literal_eval(f.read())
CLIENT = TOKENS['CLIENT']
CLIENT_ID = CLIENT['CLIENT_ID']
CLIENT_SECRET = CLIENT['CLIENT_SECRET']
REDIRECT_URI = 'http://127.0.0.1:8080/'

In [5]:
server = OAuth2Server(CLIENT_ID,CLIENT_SECRET)
server.browser_authorize()

profile = server.fitbit.user_profile_get()
TOKENS = server.fitbit.client.session.token

[14/Mar/2024:14:00:00] ENGINE Listening for SIGTERM.
[14/Mar/2024:14:00:00] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[14/Mar/2024:14:00:00] ENGINE Set handler for console events.
[14/Mar/2024:14:00:00] ENGINE Started monitor thread 'Autoreloader'.
[14/Mar/2024:14:00:01] ENGINE Serving on http://127.0.0.1:8080
[14/Mar/2024:14:00:01] ENGINE Bus STARTED


127.0.0.1 - - [14/Mar/2024:14:00:10] "GET /?code=ee319272e94876e364a1a5e0cb6a535203f232da&state=whqKjUrMySDHbwCAjVI3PQVOlec8DP HTTP/1.1" 200 122 "" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"
127.0.0.1 - - [14/Mar/2024:14:00:10] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/?code=ee319272e94876e364a1a5e0cb6a535203f232da&state=whqKjUrMySDHbwCAjVI3PQVOlec8DP" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"


[14/Mar/2024:14:00:11] ENGINE Bus STOPPING
[14/Mar/2024:14:00:11] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[14/Mar/2024:14:00:11] ENGINE Removed handler for console events.
[14/Mar/2024:14:00:11] ENGINE Stopped thread 'Autoreloader'.
[14/Mar/2024:14:00:11] ENGINE Bus STOPPED
[14/Mar/2024:14:00:11] ENGINE Bus EXITING
[14/Mar/2024:14:00:11] ENGINE Waiting for child threads to terminate...
[14/Mar/2024:14:00:11] ENGINE Bus EXITED
[14/Mar/2024:14:00:11] ENGINE Waiting for thread Thread-18.


In [6]:
ACCESS_TOKEN = TOKENS['access_token']
REFRESH_TOKEN = TOKENS['refresh_token']
EXPIRES_AT = TOKENS['expires_at']

In [7]:
client = Fitbit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    access_token=ACCESS_TOKEN,
    refresh_token=REFRESH_TOKEN,
    expires_at=EXPIRES_AT,
    redirect_uri=REDIRECT_URI
)

In [8]:
yesterday = date.today() - timedelta(days=1)

In [9]:
sleep = client.sleep(date=yesterday)['sleep'][0]
bedtime = pd.to_datetime(sleep['startTime'])
waketime = pd.to_datetime(sleep['endTime'])
sleep_duration = round(sleep['minutesAsleep'] / 60,2)

### Exercise data

In [19]:
example = client.intraday_time_series('activities/heart',base_date=yesterday)['activities-heart-intraday']['dataset']

In [20]:
example

[{'time': '00:00:00', 'value': 112},
 {'time': '00:01:00', 'value': 119},
 {'time': '00:02:00', 'value': 123},
 {'time': '00:03:00', 'value': 122},
 {'time': '00:04:00', 'value': 131},
 {'time': '00:05:00', 'value': 146},
 {'time': '00:06:00', 'value': 118},
 {'time': '00:07:00', 'value': 112},
 {'time': '00:08:00', 'value': 112},
 {'time': '00:09:00', 'value': 113},
 {'time': '00:10:00', 'value': 111},
 {'time': '00:11:00', 'value': 112},
 {'time': '00:12:00', 'value': 105},
 {'time': '00:13:00', 'value': 104},
 {'time': '00:14:00', 'value': 89},
 {'time': '00:15:00', 'value': 102},
 {'time': '00:16:00', 'value': 105},
 {'time': '00:17:00', 'value': 111},
 {'time': '00:18:00', 'value': 108},
 {'time': '00:19:00', 'value': 110},
 {'time': '00:20:00', 'value': 120},
 {'time': '00:21:00', 'value': 122},
 {'time': '00:22:00', 'value': 122},
 {'time': '00:23:00', 'value': 107},
 {'time': '00:24:00', 'value': 97},
 {'time': '00:25:00', 'value': 92},
 {'time': '00:26:00', 'value': 91},
 {'ti

In [23]:
hr_df = pd.DataFrame(example)
hr_df['time'] = pd.to_datetime(hr_df['time'],format='%H:%M:%S').dt.time
hr_df['time'] = hr_df.apply(lambda row: datetime.combine(yesterday,row['time']),axis=1)

In [24]:
hr_df

,time,value
0,2024-03-08 00:00:00,112
1,2024-03-08 00:01:00,119
2,2024-03-08 00:02:00,123
3,2024-03-08 00:03:00,122
4,2024-03-08 00:04:00,131
...,...,...
1435,2024-03-08 23:55:00,76
1436,2024-03-08 23:56:00,71
1437,2024-03-08 23:57:00,73
1438,2024-03-08 23:58:00,73


In [26]:
doubled = pd.concat([hr_df,hr_df],ignore_index=True)

## Dependent variables
### Heart rate

In [ ]:
hr = client.time_series('activities/heart',base_date=yesterday,period='1d')['activities-heart-intraday']['dataset']
hr_df = pd.DataFrame(hr)
hr_df['time'] = pd.to_datetime(hr_df['time'],format='%H:%M:%S').dt.time
hr_df['datetime'] = hr_df.apply(lambda row: datetime.combine(yesterday,row['time']),axis=1)
hr_df['30MinRollingAvg'] = hr_df['value'].rolling(window=30).mean()
rhr = round(hr_df['30MinRollingAvg'].min(),1)

In [ ]:
activities_data = client.activities(date=yesterday)

In [ ]:
summary = activities_data['summary']

In [ ]:
steps = summary['steps']

In [ ]:
sedentary_hours = round(summary['sedentaryMinutes'] / 60,2)

Maximum Heart Rate (MHR) is 208 - Age * 0.7 

In [ ]:
mhr = 208 - 31 * 0.7

Moderate intensity exercise = 50% - 70% of MHR
Vigorous intensity exercise = 70% - 85% of MHR

In [ ]:
mod_hr = (round(0.5*mhr),round(0.7*mhr))

In [ ]:
vig_hr = (round(0.7*mhr), round(0.85*mhr))

In [ ]:
activities_list = client.activities_list()['categories']

In [ ]:
keep = ['Yoga','Dancing','Sports and Workouts','Walking']
keep_activities = []
for activity in activities_list:
    if activity['name'] in keep:
        keep_activities.append(activity)

In [ ]:
data = {'date':yesterday, 'sleep_duration': sleep_duration, 'bedtime':bedtime, 'waketime':waketime, 'rhr':rhr, 'steps':steps, 'sedentary_hours':sedentary_hours}

In [ ]:
row = pd.Series(data)

In [ ]:
row

In [ ]:
df

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint as pp

In [ ]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

In [ ]:
creds = ServiceAccountCredentials.from_json_keyfile_name('creds.json',scope)
client = gspread.authorize(creds)

In [ ]:
sheet = client.open('quantified_ed').sheet1

In [ ]:
row = yesterday.strftime("%m/%d/%Y"), sleep_duration, bedtime.strftime("%m/%d/%Y, %H:%M:%S"), waketime.strftime("%m/%d/%Y, %H:%M:%S"), rhr, steps, sedentary_hours

In [ ]:
sheet.insert_row(row,2)

In [ ]:
hr_df

In [ ]:
import plotly.express as px
fig = px.line(hr_df,x='datetime',y='value')
fig.add_vline(x = waketime, line_color='green', line_width=1, line_dash="dash")
fig.add_vline(x = bedtime, line_color='purple', line_width=1, line_dash="dash")
fig.add_
fig.show()

In [ ]:
hr_df

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,7))
plt.plot(hr_df['datetime'],hr_df['value'], label='Original Heart Rate', color='red')
plt.plot(hr_df['datetime'],hr_df['30MinRollingAvg'], label='30-Point Rolling Average', color='blue', linestyle='--')
end_highlight = hr_df['datetime'][411]
start_highlight = hr_df['datetime'][381]
plt.axvspan(start_highlight,end_highlight,color='yellow',alpha=0.3)

plt.axvline(x=waketime,color='green',linestyle='--', label='Wake time')
plt.axvline(x=bedtime,color='purple',linestyle='--', label='Sleep time')

# Customizing the plot
plt.title('Heart Rate and Rolling Average Over 24 Hours')
plt.xlabel('Time')
plt.ylabel('Heart Rate (bpm)')
plt.legend()
plt.xticks(rotation=45)  # Rotate labels if needed
plt.tight_layout()

# Show plot
plt.show()

In [ ]:
rhr

## Get weight & Body fat

In [80]:
body_weights = []

In [81]:
base_date = date.today() - timedelta(days=365)

In [83]:
for month in range(1,13):
    new_date = base_date + timedelta(days=30*month)
    body_weight = client.get_bodyweight(base_date = new_date, period='1m')
    body_weights.append(body_weight)

In [112]:
flattened_data = [entry for sublist in body_weights for entry in sublist['weight']]

In [114]:
df = pd.DataFrame(flattened_data)

In [117]:
df.to_csv('output/csv/weight.csv',index=False)

In [118]:
df.to_pickle('output/pickle/weight.pkl')